<h1>Yahoo Finance Ticker Statistics Page Scraper</h1>
<h3>This script takes a list of tickers and scraps its info from the yahoo finance statistics page.</h3>

<a href = "https://finance.yahoo.com/quote/TSLA/key-statistics?p=TSLA"><img src="sample.JPG"></a>

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
import pandas as pd
from datetime import date


In [4]:
tickers = ['CRWD',
           'TEAM',
           'ZS',
           'NET',
           'SNOW',
           'HUBS',
           'U',
           'DDOG',
           'DOCN',
           'ZM',
           'SEMR',
           'MDB',
           'SHOP']

hashtable = {}

<h2>Retrieving the columns and renaming accordingly</h2>

<ol>
    <li>Create a driver to access a statistics page for a ticker to extract the metric titles of all the tables.</li>
    <li>Compile all the metric titles into a single dataframe through concatenation.</li>
</ol>

In [5]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.maximize_window() 
driver.get('https://finance.yahoo.com/quote/TSLA/key-statistics?p=TSLA')

dfs = []

#find all tables in the div container using xpath
tables = driver.find_elements(by='xpath', 
                              value='//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]//table')

#iterate through each table and rows and get the 1st td (table data), save into a list
#save each table into a list for retrieval later
for table in tables:
    spec_name = []
    for row in table.find_elements(by='xpath', value=".//tr"):
        box = row.find_elements(by='xpath', value = './/td')
        spec_name.append(box[0].get_attribute('textContent'))
    df = pd.DataFrame({"Metrics":spec_name})
    dfs.append(df)

driver.quit()

#combine all the dataframes from the list of dataframes
all_stats_df = pd.DataFrame()
for n in range(len(dfs)):
    all_stats_df = pd.concat([all_stats_df, dfs[n]])

all_stats_df.head(10)

,Metrics
0,Market Cap (intraday)
1,Enterprise Value
2,Trailing P/E
3,Forward P/E
4,PEG Ratio (5 yr expected)
5,Price/Sales (ttm)
6,Price/Book (mrq)
7,Enterprise Value/Revenue
8,Enterprise Value/EBITDA
0,Beta (5Y Monthly)


<h2>Tidying up the dataframe</h2>

Notice that there are annotation numbers at the back of certain rows, there are also rows with dates.

<ol>
    <li>Reset the index.</li>
    <li>Use Regex to remove annotation and dates.</li>
</ol>

In [6]:
all_stats_df.reset_index(inplace=True)
all_stats_df.drop(['index'],axis = 1, inplace=True)
all_stats_df['Metrics'].replace(regex={r'[0-9]$': ''}, inplace = True) 
#removes the annotations appearing at the end of rows

display(all_stats_df.iloc[23:28])
all_stats_df.iloc[23:28,0].replace(regex={r'(\(.+\))': ''}, inplace = True) 
#remove the dates under rows 23-27

display(all_stats_df.iloc[23:28])

all_stats_df.head(10)

,Metrics
23,"Shares Short (Aug 14, 2022)"
24,"Short Ratio (Aug 14, 2022)"
25,"Short % of Float (Aug 14, 2022)"
26,"Short % of Shares Outstanding (Aug 14, 2022)"
27,"Shares Short (prior month Jul 14, 2022)"


,Metrics
23,Shares Short
24,Short Ratio
25,Short % of Float
26,Short % of Shares Outstanding
27,Shares Short


,Metrics
0,Market Cap (intraday)
1,Enterprise Value
2,Trailing P/E
3,Forward P/E
4,PEG Ratio (5 yr expected)
5,Price/Sales (ttm)
6,Price/Book (mrq)
7,Enterprise Value/Revenue
8,Enterprise Value/EBITDA
9,Beta (5Y Monthly)


<h2>Scraping all the ticker stats and storing the dataframes into a hashtable</h2>
<ol>
    <li>Access each ticker based on list fo Tickers input at start.</li>
    <li>Similar to previous run, save the table contents but place each ticker stats into a hashtable for retrieval later.</li>
</ol>

In [7]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.maximize_window() 

for i in range(len(tickers)):
    driver.get(f'https://finance.yahoo.com/quote/{tickers[i]}/key-statistics?p={tickers[i]}')
    #print(f'https://finance.yahoo.com/quote/{tickers[i]}/key-statistics?p={tickers[i]}')
    
    dfs = []

    tables = driver.find_elements(by='xpath', value='//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]//table')
    for table in tables:
        spec_item = []
        for row in table.find_elements(by='xpath', value=".//tr"):
            box = row.find_elements(by='xpath', value = './/td')
            spec_item.append(box[1].get_attribute('textContent'))
        df = pd.DataFrame({f"{tickers[i]}" :spec_item})
        dfs.append(df)
    
    dummy_df = pd.DataFrame()
    for n in range(len(dfs)):
        dummy_df = pd.concat([dummy_df, dfs[n]])
    
    hashtable[f'{tickers[i]}_stats'] = dummy_df
    hashtable[f'{tickers[i]}_stats'].reset_index(inplace=True)
    hashtable[f'{tickers[i]}_stats'].drop(['index'],axis = 1, inplace=True)

driver.quit()
hashtable[f'{tickers[i]}_stats'].head()

,SHOP
0,38.19B
1,32.43B
2,N/A
3,1.43k
4,N/A


<h2>Combining all the ticker stats</h2>

In [8]:
for i in range(len(tickers)):
    all_stats_df = pd.concat([all_stats_df, hashtable[f'{tickers[i]}_stats']], axis = 1)

all_stats_df

,Metrics,CRWD,TEAM,ZS,NET,SNOW,HUBS,U,DDOG,DOCN,ZM,SEMR,MDB,SHOP
0,Market Cap (intraday),39.94B,59.25B,20.59B,19.23B,54.88B,15.14B,12.17B,30.68B,3.72B,24.05B,1.60B,16.35B,38.19B
1,Enterprise Value,38.40B,59.11B,19.93B,19.15B,51.17B,14.64B,12.23B,29.78B,4.02B,18.62B,1.42B,15.74B,32.43B
2,Trailing P/E,N/A,N/A,N/A,N/A,N/A,N/A,N/A,9.69k,N/A,24.94,N/A,N/A,N/A
3,Forward P/E,144.93,140.85,136.99,625.00,1.11k,98.04,N/A,92.59,34.72,21.93,N/A,N/A,1.43k
4,PEG Ratio (5 yr expected),3.55,4.63,2.73,N/A,5.55,2.56,N/A,1.58,N/A,11.09,N/A,N/A,N/A
5,Price/Sales (ttm),21.60,21.01,20.91,23.19,32.58,9.80,9.73,23.30,8.30,5.76,7.07,15.35,7.64
6,Price/Book (mrq),32.75,176.56,38.73,33.12,10.11,17.38,5.33,25.25,77.13,4.01,7.34,25.37,4.38
7,Enterprise Value/Revenue,20.94,21.09,20.55,23.57,31.24,9.60,10.03,21.81,8.17,4.34,6.36,14.54,6.48
8,Enterprise Value/EBITDA,-988.16,-132.89,-74.06,-119.54,-76.57,-618.97,-23.71,469.06,62.70,20.40,-281.86,-49.65,-16.81
9,Beta (5Y Monthly),1.28,1.01,1.03,1.22,N/A,1.60,N/A,1.15,N/A,-0.61,N/A,0.95,1.80


In [11]:
t_all_stats_df = all_stats_df.transpose()
t_all_stats_df.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
Metrics,Market Cap (intraday),Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue,Enterprise Value/EBITDA,Beta (5Y Monthly),...,Diluted EPS (ttm),Quarterly Earnings Growth (yoy),Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm),Levered Free Cash Flow (ttm)
CRWD,39.94B,38.40B,N/A,144.93,3.55,21.60,32.75,20.94,-988.16,1.28,...,-0.79,N/A,2.15B,9.28,772.05M,69.12,1.83,4.75,642.21M,588.47M
TEAM,59.25B,59.11B,N/A,140.85,4.63,21.01,176.56,21.09,-132.89,1.01,...,-2.86,N/A,1.47B,5.77,1.31B,391.68,1.19,1.32,883.5M,253.75M
ZS,20.59B,19.93B,N/A,136.99,2.73,20.91,38.73,20.55,-74.06,1.03,...,-2.67,N/A,1.66B,11.69,1.01B,189.10,2.24,3.75,263.51M,425.37M
NET,19.23B,19.15B,N/A,625.00,N/A,23.19,33.12,23.57,-119.54,1.22,...,-0.83,N/A,1.64B,5.01,1.57B,270.03,5.45,1.77,36.48M,91.88M


In [19]:
col = all_stats_df['Metrics']
t_all_stats_df.columns = col
t_all_stats_df.drop(index='Metrics', inplace= True)
t_all_stats_df

Metrics,Market Cap (intraday),Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue,Enterprise Value/EBITDA,Beta (5Y Monthly),...,Diluted EPS (ttm),Quarterly Earnings Growth (yoy),Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm),Levered Free Cash Flow (ttm)
CRWD,39.94B,38.40B,N/A,144.93,3.55,21.60,32.75,20.94,-988.16,1.28,...,-0.79,N/A,2.15B,9.28,772.05M,69.12,1.83,4.75,642.21M,588.47M
TEAM,59.25B,59.11B,N/A,140.85,4.63,21.01,176.56,21.09,-132.89,1.01,...,-2.86,N/A,1.47B,5.77,1.31B,391.68,1.19,1.32,883.5M,253.75M
ZS,20.59B,19.93B,N/A,136.99,2.73,20.91,38.73,20.55,-74.06,1.03,...,-2.67,N/A,1.66B,11.69,1.01B,189.10,2.24,3.75,263.51M,425.37M
NET,19.23B,19.15B,N/A,625.00,N/A,23.19,33.12,23.57,-119.54,1.22,...,-0.83,N/A,1.64B,5.01,1.57B,270.03,5.45,1.77,36.48M,91.88M
SNOW,54.88B,51.17B,N/A,1.11k,5.55,32.58,10.11,31.24,-76.57,N/A,...,-2.10,N/A,3.95B,12.42,241.76M,4.45,3.21,17.05,343.48M,608.1M
HUBS,15.14B,14.64B,N/A,98.04,2.56,9.80,17.38,9.60,-618.97,1.60,...,-1.36,N/A,1.25B,25.95,746.21M,85.63,2.31,18.19,260.98M,225.78M
U,12.17B,12.23B,N/A,N/A,N/A,9.73,5.33,10.03,-23.71,N/A,...,-2.10,N/A,1.75B,5.88,1.82B,79.59,3.44,7.66,62.54M,168.05M
DDOG,30.68B,29.78B,9.69k,92.59,1.58,23.30,25.25,21.81,469.06,1.15,...,0.00,N/A,1.7B,5.38,811.29M,66.77,3.31,3.84,403.52M,353.84M
DOCN,3.72B,4.02B,N/A,34.72,N/A,8.30,77.13,8.17,62.70,N/A,...,-0.32,N/A,1.17B,12.03,1.47B,"3,037.97",16.59,0.50,148.57M,56.85M
ZM,24.05B,18.62B,24.94,21.93,11.09,5.76,4.01,4.34,20.40,-0.61,...,3.27,-85.60%,5.52B,18.55,97.94M,1.63,3.44,20.10,1.39B,1.45B


<h2>Further improvements</h2>
To convert values in table into proper floats and datetime formats for possible ML applications.

<h2>Exporting to a dated csv file</h2>

In [20]:
current_datetime = date.today().isoformat()
t_all_stats_df.to_csv(f'saas_{current_datetime}.csv')
print(f'Saved to file: saas_{current_datetime}.csv')

Saved to file: saas_2022-09-08.csv
